In [179]:
import cvxpy as cp
import numpy as np

### HW 8, problem 2

In [1]:
C_X, C_D, C_K = 1, 3, 5
x_k_vals, k_vals = list(range(6)), list(range(5))
capacity = len(x_k_vals) - 1
n_rows, n_cols = len(x_k_vals), len(k_vals)
res = [[None for _ in range(n_cols)] for _ in range(n_rows)]
d_dist = [[0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.0, 0.0],
          [0.5, 0.25, 0.5, 0.5],
          [0.5, 0.75, 0.5, 0.5]]

for j in range(n_cols - 1, -1, -1):
    C_U = 0 if j == 0 else 2

    for i in range(n_rows):
        # Base case
        if j == n_cols - 1:
            v_k, u_k = C_K * i, 0

        # All other cases (use dynamic programming)
        else:
            e_d_k1 = 0

            for d in range(len(d_dist)):
                e_d_k1 += d * d_dist[d][j]

            possible_v_k_vals = []

            for u in x_k_vals:
                if u + i > capacity:
                    continue

                g = (C_U * u) + (C_X * i) + (C_D * max(0, e_d_k1 - i - u))
                e_v_k1 = 0

                for d in range(len(d_dist)):
                    x_k1 = max(0, i + u - d)
                    e_v_k1 += d_dist[d][j] * res[x_k1][j + 1][0]

                possible_v_k_vals.append((g + e_v_k1, u))

            v_k, u_k = min(possible_v_k_vals, key=lambda x: x[0])

        res[i][j] = (v_k, u_k)

for row in res:
    print(row)


[(14.25, 5), (16.75, 3), (11.0, 2), (5.5, 2), (0, 0)]
[(15.25, 4), (15.75, 2), (10.0, 1), (4.5, 1), (5, 0)]
[(16.25, 3), (14.75, 1), (9.0, 0), (3.5, 0), (10, 0)]
[(17.25, 2), (13.75, 0), (8.0, 0), (5.5, 0), (15, 0)]
[(18.25, 1), (13.75, 0), (8.0, 0), (11.5, 0), (20, 0)]
[(19.25, 0), (13.75, 0), (9.5, 0), (17.5, 0), (25, 0)]


### HW 9, problem 2

In [99]:
# Given parameters
l = 1
g = 9.8
m = 0.3
T = 9

# Define matrices A, B, and C
A = np.array([[0, 1], [g / l, 0]])
B = np.array([[0], [1 / (m * l**2)]])
C = np.array([[1, 0]])

# Define the initial condition
x1 = np.array([1, 2])

# Define the decision variables
x = cp.Variable((2, T + 1))
u = cp.Variable((1, T))

# Define the cost function
cost = 0

# Define the constraints
constraints = [x[:, T] == 0, x[:, 0] == x1]

for k in range(T):
    cost += cp.sum_squares(C @ x[:, k]) + 3 * cp.sum_squares(u[:, k])
    constraints += [x[:, k + 1] == A @ x[:, k] + B @ u[:, k]]

# Formulate the optimization problem
problem = cp.Problem(cp.Minimize(cost), constraints)

# Solve the problem
problem.solve()

# Extract the optimal input sequence
optimal_u = u.value
print("Optimal input sequence u:", optimal_u.flatten())

Optimal input sequence u: [-2.91053553e+00 -5.82107111e+00 -2.85857946e-01 -5.71715917e-01
 -2.80755095e-02 -5.61512231e-02 -2.75744326e-03 -5.51695963e-03
 -2.70923807e-04]


In [94]:
states = x.value

states.T

array([[1.00000000e+00, 2.00000000e+00],
       [2.00000000e+00, 9.82148956e-02],
       [9.82148956e-02, 1.96429618e-01],
       [1.96429618e-01, 9.64615722e-03],
       [9.64615722e-03, 1.92905368e-02],
       [1.92905368e-02, 9.47309029e-04],
       [9.47309029e-04, 1.87651688e-03],
       [1.87651688e-03, 9.21509548e-05],
       [9.21509548e-05, 2.98434304e-20],
       [1.49208479e-20, 1.00384470e-24]])

In [95]:
cost.value

133.354617846427

In [117]:
costs = []

for _ in range(1000):
    # u = np.random.uniform(-6, 0, T)
    u = [-1] * 9
    cost = 0
    x = np.array([1, 2])

    for k in range(T):
        curr_u = u[k]
        foo = C @ x
        foo1 = (foo.T @ foo)
        cost += foo1 ** 2 + 3 * curr_u ** 2
        x = A @ x + B @ np.array([curr_u])

    costs.append(cost)

print(np.array(costs).mean())

1083725212461752.6


### HW 9, problem 3

In [29]:
A = np.array([[1, 2, 0], [-4, 1, 0], [0, -1, 3]])
B = np.array([[1, 2], [0, -1], [-1, 0]])
C = np.array([[-1, 4, 0], [2, 0, 0]])
R = np.array([[1, 0], [0, 3]])
x = np.array([-13, 25, 20])
K = 5
v_vals = []
p_vals = [0] * (K + 1)
p_vals[-1] = np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
x_vals = []

# Calculate the all of the P values first
for i in range(K - 1, -1, -1):
    p_k1 = p_vals[i + 1]
    p_k = C.T @ C + A.T @ p_k1 @ A - A.T @ p_k1 @ B @ np.linalg.inv(R + B.T @ p_k1 @ B) @ B.T @ p_k1 @ A
    p_vals[i] = p_k

# Use the P values to calculate all of the V values (while also keeping track of the state x)
for i in range(K):
    x_vals.append(x)
    p_k, p_k1 = p_vals[i], p_vals[i + 1]
    v_k = x.T @ p_k @ x
    v_vals.append(v_k)
    u = -x @ A.T @ p_k1 @ B @ np.linalg.inv(R + B.T @ p_k1 @ B)
    print(u)
    x = A @ x + B @ u

[-121.91400819   49.27368737]
[-19.43317743 -25.05705566]
[2.87271787 0.44747956]
[0.1636993  0.37159108]
[0. 0.]


In [25]:
v_vals

[48112.797348392196,
 12521.08315224423,
 54.57251183276591,
 2.1083493081315248,
 0.0390220633416327]

In [26]:
x_vals

[array([-13,  25,  20]),
 array([ 13.63336656,  27.72631263, 156.91400819]),
 array([-4.61296933e-01, -1.75009794e+00,  4.62448889e+02]),
 array([-1.93815822e-01, -3.52389773e-01,  1.38622405e+03]),
 array([8.28608974e-03, 5.12824373e-02, 4.15886084e+03])]

### HW 9, problem 4 (exercise 17.1 in the book)

In [41]:
def ttt_addX(Sk):
    XplayMasks = np.array([0b010000000000000000,
                           0b000100000000000000,
                           0b000001000000000000,
                           0b000000010000000000,
                           0b000000000100000000,
                           0b000000000001000000,
                           0b000000000000010000,
                           0b000000000000000100,
                           0b000000000000000001], dtype=np.int32)

    print(Sk.shape)
    newS = np.zeros((Sk.shape[0], len(XplayMasks)), dtype=np.int32)
    invalid = np.zeros_like(newS, dtype=bool)

    for slot in range(len(XplayMasks)):
        mask = XplayMasks[slot]
        newS[:, slot] = np.bitwise_or(Sk, mask)
        invalid[np.bitwise_and(Sk, mask + 2 * mask) != 0, slot] = True

    XwinMasks = np.array([0b010101000000000000,
                          0b000000010101000000,
                          0b000000000000010101,
                          0b010000010000010000,
                          0b000100000100000100,
                          0b000001000001000001,
                          0b010000000100000001,
                          0b000001000100000100,
                          0b000100000001000000], dtype=np.int32)

    won = np.zeros_like(newS, dtype=bool)

    for i in range(len(XwinMasks)):
        won = np.bitwise_or(won, np.bitwise_and(newS, XwinMasks[i]) == XwinMasks[i])

    return newS, won, invalid

import numpy as np

def ttt_addO(Sk):
    OplayMasks = np.array([0b100000000000000000,
                          0b001000000000000000,
                          0b000010000000000000,
                          0b000000100000000000,
                          0b000000001000000000,
                          0b000000000010000000,
                          0b000000000000100000,
                          0b000000000000001000,
                          0b000000000000000010], dtype=np.int32)

    newS = np.zeros((Sk.shape[0], len(OplayMasks)), dtype=np.int32)
    invalid = np.zeros_like(newS, dtype=bool)

    for slot in range(len(OplayMasks)):
        mask = OplayMasks[slot]
        newS[:, slot] = np.bitwise_or(Sk, mask)
        invalid[np.bitwise_and(Sk, mask + mask//2) != 0, slot] = True

    OwinMasks = np.array([0b101010000000000000,
                          0b000000101010000000,
                          0b000000000000101010,
                          0b100000100000100000,
                          0b001000001000001000,
                          0b000010000010000010,
                          0b100000001000000010,
                          0b000010001000100000], dtype=np.int32)

    won = np.zeros_like(newS, dtype=bool)

    for i in range(len(OwinMasks)):
        won = np.bitwise_or(won, np.bitwise_and(newS, OwinMasks[i]) == OwinMasks[i])

    return newS, won, invalid

def ttt_states(S0):
    K = 9
    S = [None] * (K + 1)
    S[0] = S0

    for k in range(K):
        if k % 2 == 0:
            # Player X (minimizer) plays at odd stages
            newS, won, invalid = ttt_addX(S[k])
        else:
            # Player O (minimizer) plays at even stages
            newS, won, invalid = ttt_addO(S[k])

        # Stack all states in a column vector
        newS = newS.flatten()
        won = won.flatten()
        invalid = invalid.flatten()

        # Store (unique) list of states for which the game continues
        S[k + 1] = np.unique(newS[~invalid & ~won])

    return S


S = ttt_states(np.array([0b000000000000000000, 0b000000000000000001], dtype=np.int32))
K = 9
V = [None] * (K + 1)
V[K] = np.zeros(len(S), dtype=np.int8)

for k in range(K, 0, -1):
    if k % 2 == 1:
        # Player X (minimizer) plays at odd stages
        newS, won, invalid = ttt_addX(S)
        # Convert states to indices in S{k+1} to get their costs-to-go from V{k+1}
        exists, newSndx = np.isin(newS, S)
        # Compute all possible values
        newV = np.zeros(newS.shape, dtype=np.int8)
        newV[exists] = V[k][newSndx[exists]]
        newV[won] = -1
        newV[invalid] = np.inf  # Penalize invalid actions for minimizer
        V[k - 1] = np.min(newV, axis=1)  # Pick the best for the minimizer

    else:
        # Player O (maximizer) plays at even stages
        newS, won, invalid = ttt_addO(S)
        # Convert states to indices in S{k+1} to get their costs-to-go from V{k+1}
        exists, newS = np.isin(newS, S)
        # Compute all possible values
        newV = np.zeros(newS.shape, dtype=np.int8)
        newV[exists] = V[k][newS[exists]]
        newV[won] = 1
        newV[invalid] = -np.inf  # Penalize invalid actions for maximizer
        V[k - 1] = np.max(newV, axis=1)  # Pick the best for the maximizer

S0_example = np.array([0b000000000000000000, 0b000000000000000001], dtype=np.int32)

# Display the resulting values for each stage
for k, values in enumerate(V):
    print(f"Values at stage {k}:")
    print(values)
    print()

(2,)
(128,)
(1095,)
(1501,)
(193,)
[array([0, 1], dtype=int32), array([    1,     4,     5,    16,    17,    64,    65,   256,   257,
        1024,  1025,  4096,  4097, 16384, 16385, 65536, 65537],
      dtype=int32), array([     6,      9,     18,     24,     25,     33,     36,     37,
           66,     72,     73,     96,     97,    129,    132,    133,
          144,    145,    258,    264,    265,    288,    289,    384,
          385,    513,    516,    517,    528,    529,    576,    577,
         1026,   1032,   1033,   1056,   1057,   1152,   1153,   1536,
         1537,   2049,   2052,   2053,   2064,   2065,   2112,   2113,
         2304,   2305,   4098,   4104,   4105,   4128,   4129,   4224,
         4225,   4608,   4609,   6144,   6145,   8193,   8196,   8197,
         8208,   8209,   8256,   8257,   8448,   8449,   9216,   9217,
        16386,  16392,  16393,  16416,  16417,  16512,  16513,  16896,
        16897,  18432,  18433,  24576,  24577,  32769,  32772,  32773,
 

AttributeError: 'list' object has no attribute 'shape'

### Midterm, problem 2

In [178]:
# Variables
C_X, C_D, C_K = 1, 3, 5
x_k_vals, k_vals = list(range(6)), list(range(5))
capacity = len(x_k_vals) - 1
n_rows, n_cols = len(x_k_vals), len(k_vals)
res = [[None for _ in range(n_cols)] for _ in range(n_rows)]
d_dist = [[0.0, 0.0, 0.0, 0.0],
          [0.0, 0.0, 0.5, 0.0],
          [0.5, 0.25, 0.5, 0.5],
          [0.5, 0.75, 0.0, 0.5]]

# Iterate backwards through the columns
for j in range(n_cols - 1, -1, -1):
    # Set C_U based on what column we're on
    C_U = 0 if j == 0 else 2

    for i in range(n_rows):
        # Base case
        if j == n_cols - 1:
            v_k, u_k = C_K * i, 0

        # All other cases (use dynamic programming)
        else:
            e_d_k1 = 0

            for d in range(len(d_dist)):
                e_d_k1 += d * d_dist[d][j]
            possible_v_k_vals = []

            for u in x_k_vals:
                if u + i > capacity:
                    continue

                g = (C_U * u) + (C_X * i) + (C_D * max(0, e_d_k1 - i - u))
                e_v_k1 = 0

                for d in range(len(d_dist)):
                    x_k1 = max(0, i + u - d)
                    e_v_k1 += d_dist[d][j] * res[x_k1][j + 1][0]

                possible_v_k_vals.append((g + e_v_k1, u))

            v_k, u_k = min(possible_v_k_vals, key=lambda x: x[0])

        res[i][j] = (v_k, u_k)

for row in res:
    print(row)


[(12.25, 5), (14.75, 3), (9.0, 1), (5.5, 2), (0, 0)]
[(13.25, 4), (13.75, 2), (8.0, 0), (4.5, 1), (5, 0)]
[(14.25, 3), (12.75, 1), (7.0, 0), (3.5, 0), (10, 0)]
[(15.25, 2), (11.75, 0), (7.0, 0), (5.5, 0), (15, 0)]
[(16.25, 1), (11.75, 0), (8.5, 0), (11.5, 0), (20, 0)]
[(17.25, 0), (12.0, 0), (13.5, 0), (17.5, 0), (25, 0)]


### Midterm, problem 3

In [173]:
# Time steps
K = 7

# Matrices
A = np.array([[1, 2, -1], [-4, 1, 2], [-1, -1, 3]])
B = np.array([[1, 2], [5, -1], [-1, 3]])
C = np.array([[-1, 4, 2], [2, 0, 1]])
R = np.array([[5, 0], [0, 2]])

# Initial condition
x1 = np.array([20, 20, 20])
x_vals = [x1]

# P vals
p_vals = [None] * (K + 1)
p_vals[-1] = np.zeros((3, 3))

for k in range(K - 1, -1, -1):
    p_k1 = p_vals[k + 1]
    p_k = C.T @ C + A.T @ p_k1 @ A - A.T @ p_k1 @ B @ np.linalg.inv(R + B.T @ p_k1 @ B) @ B.T @ p_k1 @ A
    p_vals[k] = p_k

# Calculate optimal actions and corresponding cost
u_vals, v_vals = [], []

for k in range(K):
    x_k = x_vals[k]
    p_k, p_k1 = p_vals[k], p_vals[k + 1]
    u_k = -x_k @ A.T @ p_k1 @ B @ np.linalg.inv(R + B.T @ p_k1 @ B)
    u_vals.append(u_k)
    x_k1 = A @ x_k + B @ u_k
    x_vals.append(x_k1)
    v_k = x_k.T @ p_k @ x_k
    v_vals.append(v_k)

print(f'Best u values:\n{np.array(u_vals)}\n')
print(f'States:\n{np.array(x_vals)}\n')
print(f'V values:\n{v_vals}\n')
print(f'Cost (V at time 0):\n{v_vals[0]}')

Best u values:
[[-7.13083451e-01 -2.40345201e+00]
 [ 2.56398148e+01 -2.88916901e-01]
 [ 3.44014487e+00  5.05455584e-01]
 [ 3.31148671e-01  8.19031988e-02]
 [ 3.04135604e-02  7.91403463e-03]
 [ 7.68389608e-03 -6.70707094e-04]
 [ 0.00000000e+00  0.00000000e+00]]

States:
[[ 2.00000000e+01  2.00000000e+01  2.00000000e+01]
 [ 3.44800125e+01 -2.11619652e+01  1.35027274e+01]
 [ 3.71533559e+00 -3.58856972e+00  6.83569517e-01]
 [ 3.05682684e-01 -3.87504291e-01  1.64556258e-04]
 [ 2.54646136e-02 -3.60657602e-02 -3.12379832e-03]
 [ 2.69852117e-03 -1.80440437e-05 -5.44170487e-03]
 [ 1.44466198e-02  1.73946490e-02 -2.87016091e-02]
 [ 7.79375270e-02 -9.77950485e-02 -1.17946096e-01]]

V values:
[32596.822744883924, 18982.727141751533, 408.9141153844364, 4.41657347483996, 0.03849057805025596, 0.0004877428875680954, 5.19526170983789e-06]

Cost (V at time 0):
32596.822744883924


In [65]:
np.array([[1, 0, 0], [-4, 1, 2], [-1, -1, 3]]).T

array([[ 1, -4, -1],
       [ 0,  1, -1],
       [ 0,  2,  3]])

In [83]:
A = np.array([[1, 0, 0], [-4, 1, 2], [-1, -1, 3]])
B = np.array([[0, 0], [5, -1], [-1, 3]])
C = np.array([[-1, 0, 0], [2, 0, 0]])

In [118]:
p_k1 = np.array([[5, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])

In [111]:
A.T @ p_k1 @ B

array([[0, 0],
       [0, 0],
       [0, 0]])

In [112]:
B.T @ p_k1 @ B

array([[0, 0],
       [0, 0]])

In [113]:
p_k1 @ B

array([[0, 0],
       [0, 0],
       [0, 0]])

In [114]:
(A.T @ p_k1 @ B) @ np.array([[1, 2], [3, 4]])

array([[0, 0],
       [0, 0],
       [0, 0]])

In [115]:
# p_k
C.T @ C + A.T @ p_k1 @ A

array([[5, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [119]:
A.T @ p_k1 @ A

array([[5, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [106]:
C.T @ C

array([[5, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [86]:
A.T @ p_k1 @ A 

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [120]:
A.T @ p_k1 @ B

array([[0, 0],
       [0, 0],
       [0, 0]])

### Midterm, problem 4

In [172]:
# Time steps and mu
K, mu = 5, 1

# Matrices
A = np.array([[1, 2, -1], [-4, 1, 2], [-1, -1, 3]])
B = np.array([[1, 5, -1]]).T
C = np.array([[-1, 4, 2], [2, 0, 1]])
E = np.array([[2, -1, -3]]).T

# Initial condition
x1 = np.array([1, 2, 1])
x_vals = [x1]

# P vals
p_vals = [None] * (K + 1)
p_vals[-1] = np.zeros((3, 3))

def _matrix_of_matrices(p_k1):
    quad1 = B.T @ p_k1 @ B
    quad1 = np.eye(quad1.shape[0]) + quad1
    quad2 = B.T @ p_k1 @ E
    quad3 = E.T @ p_k1 @ B
    quad4 = E.T @ p_k1 @ E
    quad4 = -(mu ** 2) * np.eye(quad4.shape[0]) + quad4
    top_row = np.concatenate((quad1, quad2), axis=1)
    bottom_row = np.concatenate((quad3, quad4), axis=1)

    return np.concatenate((top_row, bottom_row), axis=0)

def _vector_of_matrices(p_k1):
    top_row = B.T @ p_k1 @ A
    bottom_row = E.T @ p_k1 @ A

    return np.concatenate((top_row, bottom_row), axis=0)

for k in range(K - 1, -1, -1):
    p_k1 = p_vals[k + 1]

    # Breaking this into components since p_k is now more complex than it was in the single-player case
    first_comp = C.T @ C + A.T @ p_k1 @ A

    second_comp_col1 = A.T @ p_k1 @ B
    second_comp_col2 = A.T @ p_k1 @ E
    second_comp = np.concatenate((second_comp_col1, second_comp_col2), axis=1)

    third_comp = _matrix_of_matrices(p_k1)

    fourth_comp = _vector_of_matrices(p_k1)

    # Use the components to calculate p_k
    p_k = first_comp - second_comp @ np.linalg.inv(third_comp) @ fourth_comp
    p_vals[k] = p_k

# Calculate optimal actions and corresponding cost
u_vals, d_vals, v_vals = [], [], []

for k in range(K):
    x_k = x_vals[k]
    p_k, p_k1 = p_vals[k], p_vals[k + 1]
    opt_vals_vector = -np.linalg.inv(_matrix_of_matrices(p_k1)) @ _vector_of_matrices(p_k1) @ x_k
    u_k, d_k = opt_vals_vector
    u_k, d_k = np.array([u_k]), np.array([d_k])
    u_vals.append(u_k)
    d_vals.append(d_k)
    x_k1 = A @ x_k + B @ u_k + E @ d_k
    x_vals.append(x_k1)
    v_k = x_k.T @ p_k @ x_k
    v_vals.append(v_k)

print(f'Best u values:\n{np.array(u_vals)}\n')
print(f'Best d values:\n{np.array(d_vals)}\n')
print(f'States:\n{np.array(x_vals)}\n')
print(f'V values:\n{v_vals}\n')
print(f'Cost (V at time 0):\n{v_vals[0]}')

Best u values:
[[ 0.68689651]
 [ 5.43365473]
 [ 3.13055646]
 [-1.09880349]
 [ 0.        ]]

Best d values:
[[ 1.10808878]
 [-8.08446061]
 [-6.30609175]
 [-2.12745671]
 [ 0.        ]]

States:
[[  1.           2.           1.        ]
 [  6.90307407   2.32639377  -4.01116285]
 [  4.83175798   1.94450602  -2.44322931]
 [  1.6823723   -0.31011048   1.68176685]
 [ -5.97333243  -7.04262671  11.15421238]
 [-31.21279823  39.15912779  46.47859628]]

V values:
[229.84787597129053, 140.60390990227685, 48.913642934075895, 22.983166121936083, 0.6403575911624784]

Cost (V at time 0):
229.84787597129053
